# Sens Critique
### Pourquoi un film fonctionne ?

## Description of the exercise: 
Work with all Data tools to create your own machine learning algorithm from non existing
database. Data value from scratch !!

**Create an algorithm which predict the popularity of a movie.**

----------------------------------

To predict the popularity of the movie, I extracted information from four types of websites:

1. A website listing 111 films (https://www.senscritique.com/films/tops/top111)
2. Websites providing descriptions of each movie (e.g., /film/douze_hommes_en_colere/370894)
3. Websites with technical descriptions of each movie (e.g., /film/douze_hommes_en_colere/370894/details)
4. Websites containing the director's descriptions (e.g., /contact/Sidney_Lumet/7800)


In [19]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
from datetime import datetime
import re

url_sc = 'https://www.senscritique.com/films/tops/top111'
page_sc = requests.get(url_sc)
soup = BeautifulSoup(page_sc.content, "html.parser")

## (1) From the Top 111 films extracted:

- French titles of the movies (as a unique ID of the movie) - *french_titles*
- Ratings of the films by users of the SensCritique website (It can be an significant feature for popularity of movie algoritm - the positive rating can positively influence the popularity of the movie) - *ratings*
- Duration of the films (maybe the length of the movie can influence its popularity. E.g, if the movie is too long,fewer people might decide to watch it) - *duration*
- Genres of the films (some genres are more populare then others) - *genre*
- Link to the further description of the moive - *movie_links*
- Link to the description of the director - *director_links*



In [20]:
#website: top 111 films
french_titles = []
titles_links = []
ratings = []
durations = []
release_date = []
genres = []
director_links = []

films = soup.find_all('div', class_="ProductListItem__Wrapper-sc-1jkxxpj-1 kusRkg")
for film in films:
    french_titles.append(film.find('a', attrs={'data-testid': 'product-title'}).text) 
    ratings.append(film.find('div', attrs={'data-testid': 'Rating'}).text) 
    durations.append(film.find('span', attrs={'data-testid': 'duration'}).text)
    release_date.append(film.find('span', attrs={'data-testid': 'date-release'}).text)
    genres.append(film.find('span', attrs={'data-testid': 'genres'}).text)
    titles_links.append(film.find('a', attrs={'data-testid': 'product-title'}).get('href'))
    director_links.append(film.find('a', attrs={'data-testid': 'link'}).get('href'))
    
df_extracted_films = pd.DataFrame({'french_titles': french_titles, 
                                   'ratings': ratings, 
                                   'duration': durations, 
                                   'release_date': release_date, 
                                   'genre': genres, 
                                   'movie_links': titles_links, 
                                   'director_links': director_links})
df_extracted_films.head(2)

,french_titles,ratings,duration,release_date,genre,movie_links,director_links
0,Douze Hommes en colère (1957),8.7,1 h 36 min.,Sortie : 4 octobre 1957 (France).,"Policier, Drame",/film/douze_hommes_en_colere/370894,/contact/Sidney_Lumet/7800
1,Harakiri (1962),8.6,2 h 13 min.,Sortie : 24 juillet 1963 (France).,Drame,/film/harakiri/402373,/contact/Masaki_Kobayashi/20521


### During the cleaning process:
- I extracted the title from the title column and removed the year information in brackets.
- I converted the duration of each movie to minutes.
- I converted the date to the datetime format. If specific information about the day was unavailable, I automatically set it to the first day of the month.
- Regarding genres, as films can be described by one or many genres, I created binary variables to describe the movies. In the dataset, I distinguished 22 genres, resulting in 22 binary variables to represent the movie's genre.



In [21]:
# Map the French month names to their English counterparts
month_map = {
    'janvier': 'January',
    'février': 'February',
    'mars': 'March',
    'avril': 'April',
    'mai': 'May',
    'juin': 'June',
    'juillet': 'July',
    'août': 'August',
    'septembre': 'September',
    'octobre': 'October',
    'novembre': 'November',
    'décembre': 'December'
}

# Function to convert date strings with French month names to datetime objects
def convert_french_date_to_datetime(date_string):
    date_string = date_string.split(': ')[1].split(' (')[0].strip(' .')
    for french, english in month_map.items():
        if french in date_string:
            date_string = date_string.replace(french, english)
    try:
        return datetime.strptime(date_string, '%d %B %Y')
    except:
        return datetime.strptime(date_string, '%B %Y')
    
# Function that create dummies from the list
def create_columns_from_list(df, column_name):
    df = df.join(pd.get_dummies(df[column_name].apply(lambda x: x.split(", ")).explode(), prefix=column_name).groupby(level=0).sum())
    return df
    

# function to convert time in string to minutes
def convert_duration(date):
    if 'h' in date:
        return int(date.split(' h ')[0])*60 + int((date.split(' h ')[1]).split(' min')[0])
    else:
        return int((date.split(' min')[0]))

In [22]:
#cleaning
df_clined_films = df_extracted_films.assign(
    french_titles=df_extracted_films['french_titles'].apply(lambda x: x.split('(')[0])
).assign(
    ratings=df_extracted_films['ratings'].astype(float),
    duration_in_min=df_extracted_films['duration'].apply(lambda date: convert_duration(date)),
    release_date=df_extracted_films['release_date'].apply(lambda x: convert_french_date_to_datetime(x))
)
df_clined_films = create_columns_from_list(df_clined_films, 'genre')
df_clined_films.drop({'duration', 'genre'}, axis=1, inplace=True)
df_clined_films.head(2)

,french_titles,ratings,release_date,movie_links,director_links,duration_in_min,genre_Action,genre_Animation,genre_Arts martiaux,genre_Aventure,...,genre_Gangster,genre_Guerre,genre_Historique,genre_Muet,genre_Policier,genre_Romance,genre_Science-fiction,genre_Thriller,genre_Western,genre_Épouvante-Horreur
0,Douze Hommes en colère,8.7,1957-10-04,/film/douze_hommes_en_colere/370894,/contact/Sidney_Lumet/7800,96,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,Harakiri,8.6,1963-07-24,/film/harakiri/402373,/contact/Masaki_Kobayashi/20521,133,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
# check if there are NaNs
df_clined_films.isna().any()

french_titles               False
ratings                     False
release_date                False
movie_links                 False
director_links              False
duration_in_min             False
genre_Action                False
genre_Animation             False
genre_Arts martiaux         False
genre_Aventure              False
genre_Biopic                False
genre_Comédie               False
genre_Comédie dramatique    False
genre_Comédie romantique    False
genre_Drame                 False
genre_Fantastique           False
genre_Fantasy               False
genre_Film noir             False
genre_Gangster              False
genre_Guerre                False
genre_Historique            False
genre_Muet                  False
genre_Policier              False
genre_Romance               False
genre_Science-fiction       False
genre_Thriller              False
genre_Western               False
genre_Épouvante-Horreur     False
dtype: bool

## (2) From movie description websites I extracted: 

- Number of votes from ranking - *number_of_votes*
- Number of people who want to see the movie - *number_want_to_see*
- Number of people who added the movie as favorite - *number_added_as_favourite*
- Number of comments - *number_of_comments*
- Details of the movie (genre, original title, country of the movie, group) - *movie_details*
- Link to the technical information about the movie - *technical_info_link*

I believe that all the features above can help with predicting the popularity of the movie because they show how many people watched or want to see the movie. *Movie details* will be useful to extract information about the country of the movie. As the website is French, maybe French movies will be more popular than others.


In [24]:
film_data = {}

films_links_to_scrap = np.unique(df_extracted_films['movie_links'])
for film in films_links_to_scrap:
    url_film = 'https://www.senscritique.com' + film
    page_film = requests.get(url_film)
    soup_film = BeautifulSoup(page_film.content, "html.parser")
    film_name = soup_film.find('h1', class_="Text__SCTitle-sc-1aoldkr-1").text
    added_as_favourite = soup_film.find('p', class_='Text__SCText-sc-1aoldkr-0 Stats__Text-sc-1u6v943-2 gATBvI irORIr').text

    # collect pupularity data
    # as there were no specific data-testid and more then one information was showed on the website using one class I callected all information 
    # saved under this class and then I extract the one I need
    popularity_info = [feature.text for feature in soup_film.find_all('p', class_="Text__SCText-sc-1aoldkr-0 Stats__Text-sc-1u6v943-2 gATBvI hzycEu")]

    # collect titles for the website (I can extraced from them the information about number of comments)
    titles_list = [title.text for title in soup_film.find_all('p', class_="NavigationTab__WrapperTextStyled-sc-18dtd9d-7")]

    #collect film details
    film_details_list = [detail.text for detail in soup_film.find_all('span', class_="Movie__Ellipsis-sc-1tik1a1-4 KQVGi")]

    # collect website link to 'Fiche technique'
    website_list = [link.get('href') for link in soup_film.find_all('a', attrs={'data-testid': "link"}, 
                                                                    class_="Text__SCText-sc-1aoldkr-0 Link__PrimaryLink-sc-1v081j9-0 gATBvI hJEAZk")]

    film_data[film] = (popularity_info[0], popularity_info[1], added_as_favourite, titles_list[2], film_details_list, website_list[1], film)

df_specific_films_extracted = pd.DataFrame(film_data).transpose().rename(
    columns={ 
        0:'number_of_votes',
        1:'number_want_to_see',
        2:'number_added_as_favourite', 
        3:'number_of_comments',
        4: 'movie_details',
        5: 'technical_info_link',
        6: 'film_link'
        })
df_specific_films_extracted.reset_index(drop=True)
df_specific_films_extracted.head(2)


,number_of_votes,number_want_to_see,number_added_as_favourite,number_of_comments,movie_details,technical_info_link,film_link
/film/andrei_roublev/372818,4.2K,6.4K,552,Critiques : avis d'internautes (73),"[Genres : Drame, Biopic, Historique, Guerre, P...",/film/andrei_roublev/372818/details,/film/andrei_roublev/372818
/film/apocalypse_now/488421,55.6K,20.9K,4.8K,Critiques : avis d'internautes (378),"[Genres : Drame, Guerre, Groupe : Au cœur des ...",/film/apocalypse_now/488421/details,/film/apocalypse_now/488421


## During the cleaning process:
- The *number_of_votes, number_want_to_see, number_added_as_favourite* columns were modify to instead of K have a numerical values
- From *number_of_comments* column extracted the numerical values
- From the *movie_details* I extracted the infomation about the *country* of the movie


In [25]:
# function that change values to numerical 
def convert_K_to_numbers(number):
    try:
        #print(number)
        if 'K' in number:
            return (float(number.split('K')[0]))*1000
        else:
            return float(number)
    except:
        return np.nan

# function that extract country of the movie from the list
def extract_country(info_list):
    for item in info_list:
        if "Pays d'origine" in item:
            return item.split(":")[-1].strip()
        

In [26]:
df_specific_films_cleaned = df_specific_films_extracted.assign(
    number_of_votes=df_specific_films_extracted['number_of_votes'].apply(lambda x: convert_K_to_numbers(x)),
    number_want_to_see=df_specific_films_extracted['number_want_to_see'].apply(lambda x: convert_K_to_numbers(x)),
    number_added_as_favourite=df_specific_films_extracted['number_added_as_favourite'].apply(lambda x: convert_K_to_numbers(x)),
    number_of_comments=df_specific_films_extracted['number_of_comments'].apply(lambda x: int(x.split('(')[1].split(')')[0])),
    country=df_specific_films_extracted['movie_details'].apply(lambda x: extract_country(x))
).drop('movie_details', axis=1)

df_specific_films_cleaned.head(2)

,number_of_votes,number_want_to_see,number_added_as_favourite,number_of_comments,technical_info_link,film_link,country
/film/andrei_roublev/372818,4200.0,6400.0,552.0,73,/film/andrei_roublev/372818/details,/film/andrei_roublev/372818,Union Soviétique
/film/apocalypse_now/488421,55600.0,20900.0,4800.0,378,/film/apocalypse_now/488421/details,/film/apocalypse_now/488421,États-Unis


In [27]:
# check if there are NaNs
df_specific_films_cleaned[df_specific_films_cleaned['country'].isna()]
df_specific_films_cleaned['country'] = df_specific_films_cleaned['country'].fillna('États-Unis')
df_specific_films_cleaned.isna().any()

number_of_votes              False
number_want_to_see           False
number_added_as_favourite    False
number_of_comments           False
technical_info_link          False
film_link                    False
country                      False
dtype: bool


## (3) From technical details of the movie websites extracted: 
- production details about the movie - year, duration, budget (I will extract *budget* from this list) (maybe movies with the higher budget will be more popular) - *production_detais*
- information about people who were in the realisation group - director, producer, distributor  - I will extract *distributor* (some distributors can by more popular then others and then the movie can by watched more often)





In [28]:
film_tech_data = {}

films_tech_links_to_scrap = np.unique(df_specific_films_extracted['technical_info_link'])
for film in films_tech_links_to_scrap:
    url_film = 'https://www.senscritique.com' + film
    page_film = requests.get(url_film)
    soup_film_tech = BeautifulSoup(page_film.content, "html.parser")

    # collect production details
    production_details_list = [param.text for param in soup_film_tech.find_all('span', class_="Info__SimpleWrapper-sc-i8lwko-2 jdUnQI")]

    # collect realisation cast information
    realisation_cast_list = [param.text for param in soup_film_tech.find_all('span', class_="Profession__SimpleWrapper-sc-17avzra-3 kKpUHr")]

    #create ID
    film = film.split('/details')[0]
    
    film_tech_data[film] = (production_details_list, realisation_cast_list, film)

df_tech_films_extracted = pd.DataFrame(film_tech_data).transpose()
df_tech_films_extracted.rename(
    columns={
        0: 'film_budget', 
        1: 'film_distributor',
        2: 'film_link'
    }, 
    inplace=True)
df_tech_films_extracted.reset_index(drop=True)
df_tech_films_extracted.tail()

,film_budget,film_distributor,film_link
/film/satantango_le_tango_de_satan/449796,"[Titre original : Sátántangó, Année : 1994, Du...","[Réalisateur : Béla Tarr, Scénaristes : László...",/film/satantango_le_tango_de_satan/449796
/film/sherlock_junior/494192,"[Titre original : Sherlock Jr., Année : 1924, ...","[Réalisateur : Buster Keaton, Producteurs : Bu...",/film/sherlock_junior/494192
/film/soy_cuba/390558,"[Année : 1964, Durée : 2 h 21 min]","[Réalisateur : Mikhail Kalatozov, Scénaristes ...",/film/soy_cuba/390558
/film/vol_au_dessus_d_un_nid_de_coucou/447958,[Titre original : One Flew Over the Cuckoo's N...,"[Réalisateur : Miloš Forman, Scénaristes : Law...",/film/vol_au_dessus_d_un_nid_de_coucou/447958
/film/voyage_au_bout_de_l_enfer/376439,"[Titre original : The Deer Hunter, Année : 197...","[Réalisateur : Michael Cimino, Scénariste : De...",/film/voyage_au_bout_de_l_enfer/376439


## During the cleaning process:
- extracted the *film_budget* and change its currency to USD
- extracted the distributor of the movie - as there are more then one distributor I decided to save only first distributor per film

In [29]:
# function that change currenct to $
def change_currency(money):
    new_currency = None
    if money is not None:
        value = re.findall(r'\d+', money)
        value = int(''.join(value))
        ext = re.findall('[a-zA-Z$¥]+', money)[-1]
        new_currency = value
        if ext == 'roubles':
            new_currency = 0.011 * value
        elif (ext == 'Yen') or (ext == '¥'):
            new_currency = 0.0067 * value
    return new_currency

In [30]:
df_tech_films_cleaned = df_tech_films_extracted.assign(
    film_budget=df_tech_films_extracted['film_budget'].apply(lambda x: x[3] if len(x) > 3 else None),
    film_distributor=df_tech_films_extracted['film_distributor'].apply(lambda x: x[3] if len(x) > 3 else None)
)
df_tech_films_cleaned['budget_in_$'] = df_tech_films_cleaned['film_budget'].apply(lambda x: change_currency(x))
df_tech_films_cleaned['film_distributor'] = df_tech_films_cleaned['film_distributor'].apply(lambda x: x.split(': ')[-1] if x is not None else 'Other')
df_tech_films_cleaned['film_distributor'] = df_tech_films_cleaned['film_distributor'].apply(lambda x: x.split(', ')[0])

df_tech_films_cleaned = df_tech_films_cleaned.drop({'film_budget'}, axis=1)

df_tech_films_cleaned.head(2)

,film_distributor,film_link,budget_in_$
/film/andrei_roublev/372818,Baba Yaga Distribution,/film/andrei_roublev/372818,14300.0
/film/apocalypse_now/488421,United Artists,/film/apocalypse_now/488421,NaN


As there are many Nans in the *budget_in_$* column, I think that I will delete before creating the Machine Learning Model. I will leave it for now, because for this exercise it is not necessery.

In [31]:
#checking nans
df_tech_films_cleaned[df_tech_films_cleaned['budget_in_$'].isna()].count()

film_distributor    26
film_link           26
budget_in_$          0
dtype: int64

## (4) From director website extracted: 
- director name
- information about the number of people who likes the director - *director_as_favourite*
- rates of the movies that the producer created - *directed_movies_rates*

I decided to extract the popularity rate (?) from the director. I think that if the director is more popular and their movies are rated well, then there is a chance that the other movie made by this director will be popular too.

In [32]:
director_data = {}

director_links_to_scrap = np.unique(df_extracted_films['director_links'])

for director in director_links_to_scrap:
    a = []
    url_director = 'https://www.senscritique.com' + director
    page_director = requests.get(url_director)
    soup_director = BeautifulSoup(page_director.content, "html.parser")
    
    # Extract director's name
    director_name = soup_director.find('h1', class_="Text__SCTitle-sc-1aoldkr-1").text
    
    # Extract director's rating
    director_as_favourite = soup_director.find('div', attrs={'data-testid':"stats"},class_="Stats__Wrapper-sc-1u6v943-0 haoJkR").text

    # Extract scores of the movies
    directed_movies_rates = [param.text for param in soup_director.find_all('div', 
                                                                            class_="Rating__GlobalRating-sc-1b09g9c-5 hdBBeX Poster__GlobalRating-sc-yale2-7 ekGaTe globalRating", 
                                                                            attrs={'data-testid':"Rating"})]

    director_data[director] = (director_name, director_as_favourite, directed_movies_rates, director)

df_director_data_extracted = pd.DataFrame(director_data).transpose()
df_director_data_extracted.rename(
    columns={
        0: 'director', 
        1: 'director_as_favourite', 
        2: 'directed_movies_rates',
        3: 'director_data_link'
    }, 
    inplace=True)
df_director_data_extracted.reset_index(drop=True)
df_director_data_extracted.head()

,director,director_as_favourite,directed_movies_rates,director_data_link
/contact/Akira_Kurosawa/1114,Akira Kurosawa,2.7K aiment,"[8.5, 7.9, 8, 8, 8.5, 8.5, 8.4, 8.3, 6.6, 7.8]",/contact/Akira_Kurosawa/1114
/contact/Alfred_Hitchcock/2622,Alfred Hitchcock,4.3K aiment,"[8.3, 8.1, 7.5, 8.1, 8.3, 8.3, 8.3, 8.1, 7, 6....",/contact/Alfred_Hitchcock/2622
/contact/Andrei_Tarkovski/2243,Andreï Tarkovski,1.7K aiment,"[8.1, 7.7, 7.9, 8.2, 9.2, 9.1, 8.7, 8.5, 7.8, ...",/contact/Andrei_Tarkovski/2243
/contact/Ari_Aster/896473,Ari Aster,875 aiment,"[7.2, 7, 6.5, 8.2, 8.2, 7.2, 7.1, 7, 6.5, 6.7,...",/contact/Ari_Aster/896473
/contact/Bela_Tarr/14656,Béla Tarr,335 aiment,"[7.9, 7.3, 8.2, 7.4, 8.2, 7.9, 7.6, 7.4, 6.2, ...",/contact/Bela_Tarr/14656


In [33]:
#cleaning 
df_director_data_cleaned = df_director_data_extracted.assign(
    director_as_favourite=df_director_data_extracted['director_as_favourite'].apply(lambda x: (convert_K_to_numbers(x.split(' ')[0]))),
    directed_movies_rates_average=df_director_data_extracted['directed_movies_rates'].apply(lambda x: np.mean(list(map(float, x))))
).drop('directed_movies_rates', axis=1)
df_director_data_cleaned.head()


,director,director_as_favourite,director_data_link,directed_movies_rates_average
/contact/Akira_Kurosawa/1114,Akira Kurosawa,2700.0,/contact/Akira_Kurosawa/1114,8.050000
/contact/Alfred_Hitchcock/2622,Alfred Hitchcock,4300.0,/contact/Alfred_Hitchcock/2622,7.714286
/contact/Andrei_Tarkovski/2243,Andreï Tarkovski,1700.0,/contact/Andrei_Tarkovski/2243,8.118182
/contact/Ari_Aster/896473,Ari Aster,875.0,/contact/Ari_Aster/896473,7.216667
/contact/Bela_Tarr/14656,Béla Tarr,335.0,/contact/Bela_Tarr/14656,7.545455


In [34]:
#checking nans
df_director_data_cleaned.isna().any()

director                         False
director_as_favourite            False
director_data_link               False
directed_movies_rates_average    False
dtype: bool

## Final table contains columns:
- *french_titles* - title of the movie
- *ratings* - rating of the movie by website users
- *release_date* - release date of the movie
- *duration_in_min* - duration of the movie in minuts
- 22 columns with dummies for genre: *'genre_Action', 'genre_Animation', 'genre_Arts martiaux', 'genre_Aventure', 'genre_Biopic',      'genre_Comédie', 'genre_Comédie dramatique', 'genre_Comédie romantique','genre_Drame', 'genre_Fantastique', 'genre_Fantasy', 'genre_Film noir','genre_Gangster', 'genre_Guerre', 'genre_Historique', 'genre_Muet','genre_Policier', 'genre_Romance', 'genre_Science-fiction','genre_Thriller', 'genre_Western', 'genre_Épouvante-Horreur'* - genre of the movie
- *number_of_votes* - number of people who watched the movie
- *number_want_to_see* - number of people who added movie to the list to watch
- *number_added_as_favourite* - number of people who marked the movie as favourite
- *number_of_comments* - number of comments 
- *country* - country of the movie production
- *film_distributor* - french film distributor of the movie
- *budget_in_$* - budget of the movie in USD
- *director* - director of the movie
- *director_as_favourite* - number of people who marked the director as their favourite
- *directed_movies_rates_average* - average rating of the director's movies




In [35]:


#combine df_final with df_specific_films_cleaned
df_movies_final = df_clined_films.merge(
    df_specific_films_cleaned, 
    left_on='movie_links', 
    right_on='film_link', how='left'
    ).drop({'film_link','technical_info_link'}, axis=1)

#combine df_clined_film with df_director_data_cleaned
df_movies_final = df_movies_final.merge(
    df_tech_films_cleaned, 
    left_on='movie_links', 
    right_on='film_link', how='left'
    ).drop({'film_link','movie_links'}, axis=1)


#combine df_clined_film with df_director_data_cleaned
df_movies_final = df_movies_final.merge(
    df_director_data_cleaned, 
    left_on='director_links', 
    right_on='director_data_link', how='left'
    ).drop({'director_links','director_data_link'}, axis=1)

df_movies_final.head()


,french_titles,ratings,release_date,duration_in_min,genre_Action,genre_Animation,genre_Arts martiaux,genre_Aventure,genre_Biopic,genre_Comédie,...,number_of_votes,number_want_to_see,number_added_as_favourite,number_of_comments,country,film_distributor,budget_in_$,director,director_as_favourite,directed_movies_rates_average
0,Douze Hommes en colère,8.7,1957-10-04,96,0,0,0,0,0,0,...,51800.0,16400.0,6800.0,592,États-Unis,Carlotta Films,350000.0,Sidney Lumet,1600.0,7.723077
1,Harakiri,8.6,1963-07-24,133,0,0,0,0,0,0,...,8200.0,12700.0,1500.0,115,Japon,Cocinor,NaN,Masaki Kobayashi,464.0,8.000000
2,Blade Runner : The Final Cut,8.5,2007-10-05,117,0,0,0,0,0,0,...,1200.0,273.0,111.0,3,États-Unis,Other,NaN,Ridley Scott,3600.0,7.184615
3,"Le Bon, la Brute et le Truand",8.5,1968-03-08,179,0,0,0,1,0,0,...,54500.0,13600.0,5500.0,379,"Italie, Espagne, Allemagne, États-Unis",Action Cinémas / Théâtre du Temple,1200000.0,Sergio Leone,2800.0,8.258333
4,Barberousse,8.5,1978-01-04,185,0,0,0,0,0,0,...,4300.0,7700.0,833.0,88,Japon,Action Cinémas / Théâtre du Temple,NaN,Akira Kurosawa,2700.0,8.050000
